In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
from src.api import ModelTrainer  
app = ModelTrainer.initialize(stage = 0 , resume = 0 , checkname= 1 , override={'env.short_test':True})
app.go()

24-10-29 15:18:48|MOD:display     |: Model Specifics:
24-10-29 15:18:48|MOD:display     |: Start Process [Data] at Tue Oct 29 15:18:48 2024!


--Process Queue : Data + Fit + Test
--Start Training New!
--Model_name is set to lgbm_day_ShortTest!
StatusDisplay(verbosity=10) , display epoch and event information
DetailedAlphaAnalysis(use_num=avg) , record and concat each model to Alpha model instance
{'model_name': 'lgbm_day_ShortTest',
 'model_module': 'lgbm',
 'model_submodels': ['best'],
 'model_booster_type': 'lgbm',
 'model_booster_optuna': True,
 'model_booster_head': None,
 'model_data_types': ['day'],
 'model_data_labels': None,
 'random_seed': None,
 'beg_date': 20170101,
 'end_date': 20170228,
 'train_sample_method': 'sequential',
 'train_shuffle_option': 'epoch'}
{'seqlens': [{'day': 5}],
 'objective': ['mse'],
 'linear_tree': [True],
 'learning_rate': [0.3],
 'reg_lambda': [1e-05],
 'reg_alpha': [1e-07],
 'num_leaves': [31],
 'max_depth': [6],
 'min_sum_hessian_in_leaf': [1],
 'feature_fraction': [0.6],
 'bagging_fraction': [0.75],
 'force_col_wise': [True],
 'monotone_constraints': [0],
 'verbosity': 10}
try using d:

24-10-29 15:18:49|MOD:display     |: Finish Process [Data], Cost 1.6 Secs
24-10-29 15:18:49|MOD:display     |: Start Process [Fit] at Tue Oct 29 15:18:49 2024!
24-10-29 15:18:49|MOD:classes     |: First Iterance: (20170103 , 0)


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


100%|██████████| 10/10 [00:00<00:00, 10031.82it/s]
[I 2024-10-29 15:18:50,750] A new study created in RDB with name: lgbm_day_ShortTest@0@20170103@best_20241029-151850_lC5KwggtNw
[I 2024-10-29 15:18:51,794] Trial 0 finished with value: -0.004384280648082495 and parameters: {'objective': 'mse', 'learning_rate': 0.007822765118210836, 'max_depth': 4, 'num_leaves': 80, 'min_data_in_leaf': 30, 'reg_alpha': 0.00039691440835509074, 'reg_lambda': 0.018736367790460333, 'feature_fraction': 0.5, 'bagging_fraction': 0.7}. Best is trial 0 with value: -0.004384280648082495.
[I 2024-10-29 15:18:52,974] Trial 1 finished with value: -0.004087692592293024 and parameters: {'objective': 'mse', 'learning_rate': 0.09130134073493743, 'max_depth': 9, 'num_leaves': 30, 'min_data_in_leaf': 20, 'reg_alpha': 0.0014272454237359725, 'reg_lambda': 72.5905294091826, 'feature_fraction': 0.5, 'bagging_fraction': 0.7}. Best is trial 1 with value: -0.004087692592293024.
[I 2024-10-29 15:18:54,453] Trial 2 finished with v

...

,group,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
num,submodel,,,,,,,,,,,,,,,,,,,,
0,best,-2.659%,-2.453%,-1.695%,-0.178%,-1.204%,-1.461%,-0.792%,1.003%,-0.850%,0.731%,-0.845%,0.901%,-0.762%,-0.687%,-0.824%,-0.712%,-0.267%,-0.317%,-0.156%,5.189%


Grouped Return Results are saved to d:\Coding\learndl\learndl\models\lgbm_day_ShortTest\detailed_analysis\group.xlsx


,factor_name,benchmark,start,end,pf,bm,excess,annualized,mdd,te,ir,calmar,turnover,mdd_period
1,best,csi300,20170104,20170302,2.26%,1.97%,0.29%,1.81%,0.31%,2.06%,0.875,5.812,3.400,20170118-20170222
2,best,csi500,20170104,20170302,1.61%,0.66%,0.95%,5.66%,0.61%,4.12%,1.372,9.342,3.400,20170118-20170222
0,best,csi1000,20170104,20170302,-0.06%,-2.29%,2.24%,14.91%,0.23%,3.37%,4.429,64.248,3.400,20170215-20170222


24-10-29 15:20:03|MOD:timer       |: Main Process Finished! Cost 1 Minutes 15.0 Seconds


Analytic datas are saved to d:\Coding\learndl\learndl\models\lgbm_day_ShortTest\detailed_analysis\data.xlsx
Analytic plots are saved to d:\Coding\learndl\learndl\models\lgbm_day_ShortTest\detailed_analysis\plot.pdf


In [4]:
app.config.model_booster_optuna


True

In [3]:
app.model.booster

In [2]:
from src.model.model_module.util.data_transform import batch_data_flatten_x
x = batch_data_flatten_x(app.model.batch_data)
x

tensor([[-0.1205, -0.1341,  0.1064,  ..., -0.0568, -0.1158,  0.1249],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        ...,
        [-0.1390, -0.1952,  0.2181,  ..., -0.3572, -0.2161,  0.2582],
        [-0.1345, -0.1976,  0.2013,  ..., -0.2841, -0.1993,  0.2030],
        [-0.1444, -0.0942,  0.0788,  ..., -0.0839, -0.1143,  0.1303]])

In [4]:
from src.algo.boost.booster.lgbm import BoosterInput
booster_input = BoosterInput.from_tensor(x)
booster_input.y is None


True

In [12]:
booster_input.obj_flatten(booster_input.x).shape

torch.Size([1, 2816, 96])

In [10]:
booster_input.x.shape

torch.Size([2816, 1, 96])

In [14]:

app.model.booster.predict(booster_input).pred.to(x)

ValueError: Input numpy.ndarray or list must be 2 dimensional

In [13]:
app.model.booster(x)

ValueError: Input numpy.ndarray or list must be 2 dimensional

In [11]:
x.shape

torch.Size([2808, 96])

In [3]:
app.model.batch_forward()


ValueError: Input numpy.ndarray or list must be 2 dimensional

In [3]:
from src2.model.callback.test import PRED_RECORD
PRED_RECORD.all_preds[(PRED_RECORD.all_preds['submodel'] == 'best') & (PRED_RECORD.all_preds['secid'] == 688981)]

,model_num,submodel,secid,date,values


In [1]:
from src2.api import ModelHiddenExtractor
ModelHiddenExtractor.update_hidden('gru_day' , model_nums=[0] , model_submodels=['best'])



Beware! Should be at server or short_test, but short_test is False now!
try using d:\Coding\learndl\learndl\data\DataSet/day.20240607.pt , success!
Load  2 DataBlocks...... finished! Cost 0.14 secs
Align 2 DataBlocks...... finished! Cost 0.22 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


100%|██████████| 14/14 [00:00<00:00, 1754.83it/s]


In [ ]:
from src2.api import DataAPI , ModelTrainer
DataAPI.reconstruct_train_data()
ModelTrainer.update_models()

In [ ]:
from src.api import HiddenExtractor
extractor = HiddenExtractor('resnet_gru_30m' , model_nums=[1] , model_types=['best'])
extractor.extract_hidden('update' , deploy = True)

In [ ]:
from src.api import DataAPI , Trainer
DataAPI.reconstruct_train_data()
Trainer.update_models()